<a href="https://colab.research.google.com/github/eugeneteoh/COMP6248-Deep-Learning/blob/master/lab_5/exercise/lab5ex_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import torch
from torchvision import transforms
from torch.utils.data import Dataset
try:
    import pytorch_lightning as pl
except:
    !pip install pytorch-lightning
    import pytorch_lightning as pl

try:
    import wandb
except:
    !pip install wandb
    import wandb

pl.seed_everything(0)

Global seed set to 0


0

In [3]:
class MyDataset(Dataset):
  def __init__(self, size=5000, dim=40, random_offset=0):
        super(MyDataset, self).__init__()
        self.size = size
        self.dim = dim
        self.random_offset = random_offset

  def __getitem__(self, index):
      if index >= len(self):
          raise IndexError("{} index out of range".format(self.__class__.__name__))

      rng_state = torch.get_rng_state()
      torch.manual_seed(index + self.random_offset)

      while True:
        img = torch.zeros(self.dim, self.dim)
        dx = torch.randint(-10,10,(1,),dtype=torch.float)
        dy = torch.randint(-10,10,(1,),dtype=torch.float)
        c = torch.randint(-20,20,(1,), dtype=torch.float)

        params = torch.cat((dy/dx, c))
        xy = torch.randint(0,img.shape[1], (20, 2), dtype=torch.float)
        xy[:,1] = xy[:,0] * params[0] + params[1]

        xy.round_()
        xy = xy[ xy[:,1] > 0 ]
        xy = xy[ xy[:,1] < self.dim ]
        xy = xy[ xy[:,0] < self.dim ]

        for i in range(xy.shape[0]):
          x, y = xy[i][0], self.dim - xy[i][1]
          img[int(y), int(x)]=1
        if img.sum() > 2:
          break

      torch.set_rng_state(rng_state)
      return img.unsqueeze(0), params

  def __len__(self):
      return self.size

train_data = MyDataset()
val_data = MyDataset(size=500, random_offset=33333)
test_data = MyDataset(size=500, random_offset=99999)

In [4]:
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader

from pytorch_lightning.loggers import WandbLogger

In [5]:
trainloader = DataLoader(train_data, batch_size=128, shuffle=True, num_workers=2)
valloader = DataLoader(val_data, batch_size=128, num_workers=2)
testloader = DataLoader(test_data, batch_size=128, num_workers=2)

## Task 1.1

In [ ]:
class BaselineCNN(pl.LightningModule):
    
    def __init__(self): # input: 40 x 40
        super().__init__()

        self.conv1 = nn.Conv2d(1, 48, (3, 3), stride=1, padding=1)
        self.fc1 = nn.Linear(48 * 40 * 40, 128)
        self.fc2 = nn.Linear(128, 2)

    def forward(self, x):
        batch_size, channels, width, height = x.size()

        x = self.conv1(x) # output: 48 x 40 x 40
        x = F.relu(x)
        x = x.view(batch_size, -1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)

        return x

    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = F.mse_loss(y, y_hat)

        self.log('train_loss', loss, on_step=False, on_epoch=True, prog_bar=True)
        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = F.mse_loss(y, y_hat)

        self.log('val_loss', loss, on_step=False, on_epoch=True, prog_bar=True)


    def test_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = F.mse_loss(y, y_hat)

        self.log('test_loss', loss, on_step=False, on_epoch=True, prog_bar=True)

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters())



In [ ]:
wandb_logger = WandbLogger(name='task1.1', project='lab5', reinit=True)

In [ ]:
model = BaselineCNN()

trainer = pl.Trainer(gpus=1, max_epochs=100, deterministic=True, logger=wandb_logger)

trainer.fit(model, trainloader, valloader)
trainer.test(model, testloader)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc



  | Name  | Type   | Params
---------------------------------
0 | conv1 | Conv2d | 480   
1 | fc1   | Linear | 9.8 M 
2 | fc2   | Linear | 258   
---------------------------------
9.8 M     Trainable params
0         Non-trainable params
9.8 M     Total params
39.325    Total estimated model params size (MB)


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]



--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_loss': 11.697065353393555}
--------------------------------------------------------------------------------


[{'test_loss': 11.697065353393555}]

In [ ]:
wandb.finish()

train_loss,0.149
epoch,99
trainer/global_step,4000
_runtime,433
_timestamp,1619478176
_step,200
val_loss,10.60818
test_loss,11.69707


train_loss,█▄▃▃▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
trainer/global_step,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
_runtime,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇███
_timestamp,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇███
_step,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
val_loss,█▇▄▃▃▂▃▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
test_loss,▁


In [11]:
class BaselineCNN2(pl.LightningModule):
    
    def __init__(self): # input: 40 x 40
        super().__init__()

        self.conv1 = nn.Conv2d(1, 48, 3, stride=1, padding=1) # output: 48 x 40 x 40
        self.conv2 = nn.Conv2d(48, 48, 3, stride=1, padding=1) # output: 48 x 40 x 40
        self.global_maxpool = nn.AdaptiveMaxPool2d(1) # output: 48 x 8 x 8
        self.fc1 = nn.Linear(48, 128)
        self.fc2 = nn.Linear(128, 2)

    def forward(self, x):
        batch_size, channels, width, height = x.size()

        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = self.global_maxpool(x)
        x = x.view(batch_size, -1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)

        return x

    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = F.mse_loss(y, y_hat)

        self.log('train_loss', loss, on_step=False, on_epoch=True, prog_bar=True)
        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = F.mse_loss(y, y_hat)

        self.log('val_loss', loss, on_step=False, on_epoch=True, prog_bar=True)


    def test_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = F.mse_loss(y, y_hat)

        self.log('test_loss', loss, on_step=False, on_epoch=True, prog_bar=True)

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters())



In [12]:
wandb_logger = WandbLogger(name='task2.1', project='lab5', reinit=True)

model = BaselineCNN2()

trainer = pl.Trainer(gpus=1, max_epochs=100, deterministic=True, logger=wandb_logger)

trainer.fit(model, trainloader, valloader)
trainer.test(model, testloader)

wandb.finish()

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]



  | Name           | Type              | Params
-----------------------------------------------------
0 | conv1          | Conv2d            | 480   
1 | conv2          | Conv2d            | 20.8 K
2 | global_maxpool | AdaptiveMaxPool2d | 0     
3 | fc1            | Linear            | 6.3 K 
4 | fc2            | Linear            | 258   
-----------------------------------------------------
27.8 K    Trainable params
0         Non-trainable params
27.8 K    Total params
0.111     Total estimated model params size (MB)


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]



--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_loss': 15.634718894958496}
--------------------------------------------------------------------------------


train_loss,12.94438
epoch,99
trainer/global_step,4000
_runtime,289
_timestamp,1619512931
_step,200
val_loss,12.76206
test_loss,15.63472


train_loss,█▆▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
trainer/global_step,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
_runtime,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
_timestamp,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
_step,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
val_loss,█▅▃▃▂▂▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
test_loss,▁


In [8]:
class BetterCNN(pl.LightningModule):
    
    def __init__(self): # input: 40 x 40
        super().__init__()

        self.conv1 = nn.Conv2d(3, 48, 3, stride=1, padding=1) # output: 48 x 40 x 40
        self.conv2 = nn.Conv2d(48, 48, 3, stride=1, padding=1) # output: 48 x 40 x 40
        self.global_maxpool = nn.AdaptiveMaxPool2d(1) # output: 48 x 8 x 8
        self.fc1 = nn.Linear(48, 128)
        self.fc2 = nn.Linear(128, 2)

    def forward(self, x):
        batch_size, channels, width, height = x.size()
        idxx = torch.repeat_interleave(
            torch.arange(-20, 20, dtype=torch.float).unsqueeze(0) / 40.0,
            repeats=40, dim=0).to(x.device)
        idxy = idxx.clone().t()
        idx = torch.stack([idxx, idxy]).unsqueeze(0)
        idx = torch.repeat_interleave(idx, repeats=x.shape[0], dim=0)
        x = torch.cat([x, idx], dim=1)

        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = self.global_maxpool(x)
        x = x.view(batch_size, -1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)

        return x

    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = F.mse_loss(y, y_hat)

        self.log('train_loss', loss, on_step=False, on_epoch=True, prog_bar=True)
        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = F.mse_loss(y, y_hat)

        self.log('val_loss', loss, on_step=False, on_epoch=True, prog_bar=True)


    def test_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = F.mse_loss(y, y_hat)

        self.log('test_loss', loss, on_step=False, on_epoch=True, prog_bar=True)

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters())



In [9]:
wandb_logger = WandbLogger(name='task3.1', project='lab5', reinit=True)

model = BetterCNN()

trainer = pl.Trainer(gpus=1, max_epochs=100, deterministic=True, logger=wandb_logger)

trainer.fit(model, trainloader, valloader)
trainer.test(model, testloader)

wandb.finish()

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
wandb: Currently logged in as: eugeneteoh (use `wandb login --relogin` to force relogin)



  | Name           | Type              | Params
-----------------------------------------------------
0 | conv1          | Conv2d            | 1.3 K 
1 | conv2          | Conv2d            | 20.8 K
2 | global_maxpool | AdaptiveMaxPool2d | 0     
3 | fc1            | Linear            | 6.3 K 
4 | fc2            | Linear            | 258   
-----------------------------------------------------
28.7 K    Trainable params
0         Non-trainable params
28.7 K    Total params
0.115     Total estimated model params size (MB)


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]



--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_loss': 1.236459493637085}
--------------------------------------------------------------------------------


train_loss,1.0131
epoch,99
trainer/global_step,4000
_runtime,291
_timestamp,1619511769
_step,200
val_loss,1.1761
test_loss,1.23646


train_loss,█▃▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
trainer/global_step,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
_runtime,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
_timestamp,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
_step,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
val_loss,█▃▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁
test_loss,▁


In [13]:
import pandas as pd

In [70]:
df_runtime = pd.DataFrame({'Runtime': [pd.Timedelta("7m 13s"), pd.Timedelta("4m 50s"), pd.Timedelta("4m 52s")], 'Total Parameters': ['9.8 M', '27.8 K', '28.7 K']}, index=[1, 2, 3])
df_runtime

,Runtime,Total Parameters
1,0 days 00:07:13,9.8 M
2,0 days 00:04:50,27.8 K
3,0 days 00:04:52,28.7 K


In [71]:
print(df_runtime.to_latex(caption="Runtime and total parameters.", label="tab:properties"))

\begin{table}
\centering
\caption{Runtime and total parameters.}
\label{tab:properties}
\begin{tabular}{lrl}
\toprule
{} &         Runtime & Total Parameters \\
\midrule
1 & 0 days 00:07:13 &            9.8 M \\
2 & 0 days 00:04:50 &           27.8 K \\
3 & 0 days 00:04:52 &           28.7 K \\
\bottomrule
\end{tabular}
\end{table}

